In [1]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
import numpy
import string
import random
from sklearn import linear_model
import dateutil.parser as parser
import numpy as np
from matplotlib import pyplot as plt

In [6]:
f = open("./lthing_data/edges.txt", 'r')
count = 0
for line in f:
    if count < 10:
        print(line)
    count += 1

f.close()
count

Rodo anehan

Rodo sevilemar

Rodo dingsi

Rodo slash

RelaxedReader AnnRig

RelaxedReader bookbroke

RelaxedReader Bumpersmom

RelaxedReader DivaColumbus

RelaxedReader AnnRig

RelaxedReader bookbroke



219790

In [4]:
def parse(path):
    g = open(path, 'r')
    for l in g:
        yield eval(l)

In [7]:
count = 0
data = []

for d in parse("./lthing_data/reviews.json"):
    # filter review without rating
    if 'stars' not in d: continue
    
    """# handle normal review (majority) and 'abuse' or 'not a review' flag
    if not d['flags']:
        d['flags'] = 0
    else:
        d['flags'] = 1"""
    
    # train and test
    data.append(d)
    count += 1

print(f"Total number of reviews with a rating: {count}")
data[1]

Total number of reviews with a rating: 1387209


{'work': '12198649',
 'flags': [],
 'unixtime': 1333756800,
 'stars': 5.0,
 'nhelpful': 0,
 'time': 'Apr 7, 2012',
 'comment': 'Help Wanted: Tales of On The Job Terror from Evil Jester Press is a fun and scary read. This book is edited by Peter Giglio and has short stories by Joe McKinney, Gary Brandner, Henry Snider and many more. As if work wasnt already scary enough, this book gives you more reasons to be scared. Help Wanted is an excellent anthology that includes some great stories by some master storytellers.\nOne of the stories includes Agnes: A Love Story by David C. Hayes, which tells the tale of a lawyer named Jack who feels unappreciated at work and by his wife so he starts a relationship with a photocopier. They get along well until the photocopier starts wanting the lawyer to kill for it. The thing I liked about this story was how the author makes you feel sorry for Jack. His two co-workers are happily married and love their jobs while Jack is married to a paranoid alcoholi

In [8]:
# 8:2 train-test split
cut = int(count * 0.8)
train_data, test_data = data[:cut], data[cut:]
del data # save memory

### Populate useful data structures.

In [10]:
usersPerItem = defaultdict(set)
itemsPerUser = defaultdict(set)

# very likely to have same ratings, so use list
ratingsPerItem = defaultdict(list)
ratingsPerUser = defaultdict(list)

for d in train_data:
    u, i, r = d['user'], d['work'], d['stars']
    usersPerItem[i].add(u)
    itemsPerUser[u].add(i)
    ratingsPerItem[i].append(r)
    ratingsPerUser[u].append(r)

# calculate 2 global averages for cold-start user or book: average of each user's/book's average ratings
avgBookRating = sum( sum(ratingsPerItem[i])/len(ratingsPerItem[i]) for i in ratingsPerItem)/len(ratingsPerItem)
avgUserRating = sum( sum(ratingsPerUser[u])/len(ratingsPerUser[u]) for i in ratingsPerUser)/len(ratingsPerUser)

print(avgBookRating, avgUserRating)

3.765420038215221 4.833333333333131


# baseline model

### A relevant simple baseline that predicts rating based on the average rating given by a user, and the average rating received by a book.

In [11]:
def feature_baseline(d):
    global avgBookRating, avgUserRating
    result = [1] # bias term
    u, i = d['user'], d['work']
    if u in ratingsPerUser:
        result.append( sum(ratingsPerUser[u]) / len(ratingsPerUser[u]) )
    else:
        result.append(avgUserRating)
    if i in ratingsPerItem:
        result.append( sum(ratingsPerItem[i]) / len(ratingsPerItem[i]) )
    else:
        result.append(avgBookRating)
    return result

In [25]:
Xtrain_baseline = np.array( [feature_baseline(d) for d in train_data] )
Xtest_baseline = np.array( [feature_baseline(d) for d in test_data] )

Ytrain = np.array( [d['stars'] for d in train_data] )
Ytest = np.array( [d['stars'] for d in test_data] )

In [26]:
model_baseline = linear_model.LinearRegression()
model_baseline.fit(Xtrain_baseline, Ytrain)

Ypred_baseline = model_baseline.predict(Xtest_baseline)
Ypred_baseline

array([3.82830107, 3.38181352, 3.98358366, ..., 3.6640496 , 3.55440923,
       4.53077375])

In [22]:
def MSE(Y1, Y2):
    return np.mean( (Y1-Y2)**2 )

In [23]:
# Check MSE
MSE(Ypred_baseline, Ytest)

0.8524583817615936

In [27]:
# Compare it to a even more trivial baseline: always predict median 4.0
MSE([4.0]*len(Ytest), Ytest)

1.032778022073082

In [28]:
# Check binary error rate, see report section 2
TP = sum( np.logical_and(Ypred_baseline>=4.0, Ytest>=4.0) )
FP = sum( np.logical_and(Ypred_baseline>=4.0, Ytest<4.0) )
TN = sum( np.logical_and(Ypred_baseline<4.0, Ytest<4.0) )
FN = sum( np.logical_and(Ypred_baseline<4.0, Ytest>=4.0) )

assert TP+FP+TN+FN == len(Ytest)

Accuracy = (TP + TN) / len(Ytest)
BER = 1 - 0.5*(TP / (TP + FN) + TN / (TN + FP))
print(f"TP:{TN}, FP:{FP}, TN:{TN}, FN:{FN}")
print(f"Accuracy:{Accuracy}, BER:{BER}")

TP:85927, FP:18265, TN:85927, FN:91745
Accuracy:0.6034846922960475, BER:0.3524270181301432
